In [1]:
import os
import logging
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
logging.getLogger("tensorflow").setLevel(logging.ERROR)

import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd
import numpy as np
import time
import copy
from scipy.stats import rankdata
from tensorflow_serving.apis import input_pb2
from sklearn.linear_model import RidgeClassifier 

In [2]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    details = tf.config.experimental.get_device_details(gpu)
    print(details)

{'compute_capability': (8, 6), 'device_name': 'NVIDIA GeForce RTX 3050 Laptop GPU'}


## Load test data and model

In [3]:
df_test = pd.read_csv('test_yahoo.csv')

In [4]:
display(df_test.head())

,relevance_label,qid,1,2,3,4,5,6,7,8,...,91,92,93,94,95,96,97,98,99,100
0,0.0,22939,0.30267,0.646710,0.0,0.34678,0.89603,0.122830,0.65796,0.77336,...,0.107100,0.47405,0.0,0.00000,0.178890,0.00000,0.16873,0.0,0.054808,0.37080
1,1.0,22939,0.77818,0.099093,0.0,0.90748,0.35544,0.531800,0.47580,0.00000,...,0.798600,0.39928,0.0,0.36453,0.635170,0.46792,0.76504,0.0,0.356410,0.85208
2,1.0,22939,0.77818,0.715050,0.0,0.97981,0.44943,0.048097,0.71858,0.00000,...,0.048097,0.13551,0.0,0.64598,0.048097,0.46792,0.10065,0.0,0.104200,0.23288
3,0.0,22939,0.53862,0.279800,0.0,0.83239,0.29999,0.657750,0.41948,0.62482,...,0.570610,0.37363,0.0,0.36453,0.370790,0.00000,0.27524,0.0,0.341860,0.54412
4,0.0,22939,0.59326,0.601570,0.0,0.77684,0.00000,0.000000,0.00000,0.33855,...,0.000000,0.00000,0.0,0.87514,0.000000,0.00000,0.00000,0.0,0.155630,0.00000


In [ ]:
loaded_model = tf.saved_model.load("yahoo_ranking_model_dir/export/latest_model")

In [ ]:
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_all(examples):
    
    exam = []
    for idx, row in examples.iterrows():     

        example_dict = {
                       f'{feat_name}':_float_feature(feat_val) for 
                        feat_name, feat_val in zip(df_test.columns.tolist()[2:], row.iloc[2:].tolist())
                    }    
        
        example_dict['relevance_label'] = _int64_feature(int(row['relevance_label']))

        example_proto = tf.train.Example(features=tf.train.Features(feature=example_dict))
        exam.append(example_proto.SerializeToString())
    return exam

## LIME

In [ ]:
class Explanations:
    def __init__(self, data, sample_size=20, visible_features=20):
        self.data = data 
        self.sample_size = sample_size 
        self.kernel_width = np.sqrt(data.shape[1]) * .75
        
    def predict(self, instances): 
        tf_example_predictor = loaded_model.signatures[tf.saved_model.REGRESS_METHOD_NAME]
        scores = tf_example_predictor(tf.convert_to_tensor(instances))[tf.saved_model.REGRESS_OUTPUTS]
        return scores

    def kernel(self, d):
    #similarity or weight based on the Gaussian kernel function
        return np.sqrt(np.exp(-(d ** 2) / self.kernel_width ** 2))
    
    def empirical_sampling(self, instance_explained):
        generated_docs = []

        for t in range(0, self.sample_size):
            instance_explained = copy.copy(instance_explained)
            num_features = instance_explained.shape[0]
            total_feature_selected = np.random.randint(0, num_features - 2) 
            selected_features = np.random.randint(2, num_features, total_feature_selected)

            for sel_feat in selected_features:
                instance_explained[sel_feat] = np.random.choice(self.data.iloc[:, sel_feat], 1)[0]

            generated_docs.append(instance_explained)

        return generated_docs
    
    def lime_inverse_zscore(self, instance_explained):
        generated_docs = []

        for t in range(0, self.sample_size):
            instance_explained = copy.copy(instance_explained)
            num_features = instance_explained.shape[0] 
            total_feature_selected = np.random.randint(0, num_features - 2)
            selected_features = np.random.randint(2, num_features, total_feature_selected)

            for sel_feat in selected_features:
                mu = np.mean(self.data.iloc[:, sel_feat].values)
                sigma = np.std(self.data.iloc[:, sel_feat].values)
                z = np.random.normal(0, 1)
                instance_explained[sel_feat] = z * sigma + mu

            generated_docs.append(instance_explained)

        return generated_docs
    
    def gaussian_sampling(self, instance_explained):
        generated_docs = []
        
        unique_vals = {}
        exclude_features = []

        for i in range(2, self.data.shape[1]):
            dist = np.abs(self.data.iloc[:, i] - instance_explained[i])
            unique_val = dist[ dist < np.std(self.data.iloc[:, i])].unique()
            
            if len(unique_val) > 0: 
                unique_vals[i] = unique_val
            else: 
                exclude_features.append(i)

        for t in range(0, self.sample_size):
            instance_explained = copy.copy(instance_explained)
            num_features = instance_explained.shape[0] 
            total_feature_selected = np.random.randint(0, num_features - 2 - len(exclude_features))
            available_features = np.setxor1d(np.arange(2, num_features), exclude_features)
            selected_features = np.random.choice(available_features, total_feature_selected)
            
            for c_feat in selected_features:
                instance_explained[c_feat] = np.random.choice(unique_vals[c_feat], 1)[0]

            generated_docs.append(instance_explained)

        return generated_docs
        
    def get_exp(self, qid_data, doc_idx, sampling): 
        #qid is a pandas dataframe
        #doc_idx index instance to explain
        start = time.time()

        docs = serialize_all(qid_data) #list of tensor examples

        original_scores = self.predict(docs) #prediction of docs given query
        base_rank = rankdata([-1 * i for i in original_scores]).astype(int) - 1

        instance_explained = copy.copy(qid_data.iloc[doc_idx])
        
        #returns list of new documents samples
        if sampling == 'empirical':
            generated_docs = self.empirical_sampling(instance_explained)
        elif sampling == 'gaussian':
            generated_docs = self.gaussian_sampling(instance_explained)
        elif sampling == 'lime':
            generated_docs = self.lime_inverse_zscore(instance_explained)
        
        generated_predictions = []

        #for each sample
        for t in range(0, self.sample_size):
            temp_docs = copy.copy(docs) #copy tensors
            temp_docs[doc_idx] = serialize_all(generated_docs[t].to_frame().T)[0] #the selected instance is replaced
            genere_pred = self.predict(temp_docs)
            generated_predictions.append(genere_pred) #return tensor predict

        #base_rank = rankdata([-1 * i for i in original_scores.flatten()]).astype(int) - 1
        
        ranked_all = []
        
        for gen_pred in generated_predictions:
            ranked_all.append(rankdata([-1 * i for i in gen_pred]).astype(int) - 1)
        ranked_all = np.array(ranked_all)
        
        labels = []
        #binary labels
        for ranked in ranked_all: 
            if ranked[doc_idx] <= base_rank[doc_idx]: 
                labels.append(1)
            else:
                labels.append(0)
                
        gen_docs = []
        
        for i in range(0, self.sample_size):
            gen_docs.append(generated_docs[i].values[2:]) #all features of generated_docs
        gen_docs = np.array(gen_docs).astype(np.float32)
        
        i_explained = instance_explained.values[2:].astype(np.float32)
        distances = np.linalg.norm(gen_docs - i_explained, axis=1) #euclidean distance
        k_weights = self.kernel(distances).astype(np.float32)
        
        clf = RidgeClassifier().fit(gen_docs, labels, sample_weight=k_weights)
        end = time.time()
        
        print('Time took for explanations: {} '.format(end - start))
        
        return original_scores, clf

In [ ]:
grouped_qid = df_test.groupby('qid')
group_data = grouped_qid.get_group(23834)

idx = 32

lime = Explanations(df_test)
GAM_scores, LIME_model = lime.get_exp(group_data, idx, "empirical")

base_rank = rankdata([-1 * i for i in GAM_scores]).astype(int) - 1

sorted_lists = sorted(zip(base_rank, group_data['relevance_label'].tolist()))
sorted_second_list = [item[1] for item in sorted_lists]

print(sorted_second_list)

print(LIME_model.coef_)